In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

# 1. Carregamento do dataset
DATA_PATH = r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\Avaliacao\heart_failure_dataset.csv'
df = pd.read_csv(DATA_PATH, sep=',', na_values=["N/D", "NA"])
print(df.head())

# Exibir dimensões do dataset
print(f'Dimensões do dataset: {df.shape}')

# Verificar tipos de dados
print(f'Tipos de dados:\n{df.dtypes}')

# Verificar valores ausentes
print(f'\nValores ausentes por coluna:\n{df.isnull().sum()}')

# Estatísticas descritivas
print(f'\nEstatísticas descritivas:\n{df.describe()}')


# Função para identificar e remover outliers usando IQR
def remove_outliers(df, columns):
    outliers_count_before = {col: (df[col] < df[col].quantile(0.25) - 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))) |
                            (df[col] > df[col].quantile(0.75) + 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))) for col in columns}
    
    # Remover outliers
    for col in columns:
        IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
        lower_bound = df[col].quantile(0.25) - 1.5 * IQR
        upper_bound = df[col].quantile(0.75) + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    outliers_count_after = {col: (df[col] < df[col].quantile(0.25) - 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))) |
                           (df[col] > df[col].quantile(0.75) + 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))) for col in columns}
    
    return df, outliers_count_before, outliers_count_after

# Selecionar variáveis numéricas
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Remover outliers
df_cleaned, outliers_before, outliers_after = remove_outliers(df, numeric_columns)

#print(f'Outliers antes da remoção:\n{outliers_before}')
#print(f'Outliers após a remoção:\n{outliers_after}')



# Para variáveis categóricas, usamos a moda
for col in df_cleaned.select_dtypes(include=[object]).columns:
    df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

# Verificar se ainda há valores ausentes
print(f'Valores ausentes após imputação:\n{df_cleaned.isnull().sum()}')


# Criar faixas etárias
df_cleaned['age_group'] = pd.cut(df_cleaned['age'], bins=[0, 40, 60, 80, 100], labels=['0-40', '40-60', '60-80', '80+'])


# Exibir as primeiras linhas para verificar as novas features
print(df_cleaned[['age', 'age_group']].head())


# One-Hot Encoding com pandas
df_one_hot = pd.get_dummies(df_cleaned, drop_first=True)

# Label Encoding para variáveis categóricas
label_encoder = LabelEncoder()
cat_cols = df_cleaned.select_dtypes(include=[object]).columns
df_label_encoded = df_cleaned.copy()

for col in cat_cols:
    df_label_encoded[col] = label_encoder.fit_transform(df_label_encoded[col])

# Verificar resultados
print("One-Hot Encoding:", df_one_hot.head())
print("\nLabel Encoding:", df_label_encoded.head())


# Selecionar variáveis numéricas para escalonamento
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
df_cleaned[numeric_columns] = scaler.fit_transform(df_cleaned[numeric_columns])

# Verificar se as variáveis estão escalonadas
print(df_cleaned[numeric_columns].head())

# Verificar duplicados
duplicates = df_cleaned.duplicated().sum()
print(f'Número de duplicados: {duplicates}')

# Verificar ausência de valores nulos
print(f'\nValores ausentes finais:\n{df_cleaned.isnull().sum()}')

# Verificar colunas numéricas escalonadas
print(f'\nColunas numéricas escalonadas:\n{df_cleaned[numeric_columns].head()}')

# Verificar colunas categóricas codificadas (One-Hot ou Label Encoding)
print(f'\nColunas categóricas codificadas:\n{df_cleaned.select_dtypes(include=[object]).head()}')

# Exportar para Pandas
df_cleaned.to_csv('heart_prepared_pandas.csv', index=False)

# Exportar para Polars
import polars as pl
df_polars = pl.from_pandas(df_cleaned)
df_polars.write_csv('heart_prepared_polars.csv')

    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       